# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Create dataframe
city_data_df = pd.read_csv('../output_data/cities_data.csv', index_col=0)

city_data_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Weather Date
0,Saint Anthony,45.0205,-93.2180,26.01,68,90,9.22,US,1611994745
1,Rikitea,-23.1203,-134.9692,79.38,75,7,11.23,PF,1611994745
2,Bangui,4.3612,18.5550,75.20,83,20,5.75,CF,1611994745
3,Chicama,-7.8447,-79.1469,72.21,69,98,4.65,PE,1611994642
4,Punta Arenas,-53.1500,-70.9167,44.60,76,40,5.75,CL,1611994745


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Customize the size of the figure

figure_layout = {
    'width': '600px',
    'height': '420px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Create base map
fig = gmaps.figure(layout=figure_layout, zoom_level=2, center=(10,25))

In [5]:
# Store 'Lat' and 'Lng' into  locations 
locations = city_data_df[["Lat", "Lng"]]


# Use Humidity as the weight
humidity = city_data_df['Humidity'] 


In [6]:
# Create Humidity Heatmap layer
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity)

# Add Heatmap layer to map
fig.add_layer(heatmap_layer)

fig



Figure(layout=FigureLayout(border='1px solid black', height='420px', margin='0 auto 0 auto', padding='1px', wi…

In [7]:
heatmap_layer.max_intensity = 100
heatmap_layer.point_radius = 3
heatmap_layer.dissipating = False

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

#### Narrow down the DataFrame to find your ideal weather condition. For example:

    * A max temperature lower than 80 degrees but higher than 70.
    * Wind speed less than 10 mph.
    * Zero cloudiness.
    * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

* Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.


In [8]:
# Create filters for desire weather condition
filter_maxtemp = (city_data_df['Max Temp']<80) & (city_data_df['Max Temp'] >75)
filter_wind = (city_data_df['Wind Speed']<10)
filter_clouds = (city_data_df['Cloudiness']==0)
                   

ideal_weather_pl=city_data_df[filter_maxtemp & filter_wind & filter_clouds]

# drop any rows with nan values
ideal_weather_pl=ideal_weather_pl.dropna()

ideal_weather_pl

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Weather Date
147,Yazman,29.1212,71.7446,76.64,14,0,7.27,PK,1611994772
192,Bubaque,11.2833,-15.8333,77.68,56,0,7.92,GW,1611994782
327,Sur,22.5667,59.5289,78.58,27,0,7.31,OM,1611994811
428,Nizwá,22.9333,57.5333,78.89,22,0,6.33,OM,1611994829
516,Pokaran,26.9167,71.9167,77.04,13,0,3.09,IN,1611994846
538,Turbat,26.0023,63.0440,78.10,17,0,9.57,PK,1611994850


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = ideal_weather_pl[['City','Country','Lat','Lng']].copy()

# Add "Hotel Name" and "Hotel Address" columns to the DataFrame
## Note that we used "" to specify initial entry.
hotel_df["Hotel Name"]=""
hotel_df["Hotel Address"]=""

hotel_df

,City,Country,Lat,Lng,Hotel Name,Hotel Address
147,Yazman,PK,29.1212,71.7446,,
192,Bubaque,GW,11.2833,-15.8333,,
327,Sur,OM,22.5667,59.5289,,
428,Nizwá,OM,22.9333,57.5333,,
516,Pokaran,IN,26.9167,71.9167,,
538,Turbat,PK,26.0023,63.0440,,


In [10]:
# Set parameters to search for hotels with 5000 meters.
# set up a parameters dictionary
query_params = {
    
    "radius": 5000, 
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}


for idx, row in ideal_weather_pl.iterrows():
    # get lat,lng from df
    lat=row["Lat"] 
    lng=row["Lng"]
    
    # change location each iteration while leaving original params in place
    query_params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=query_params)
    
    # convert to json
    name_address = response.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))
    
    # Use try-except to skip any missing data
    try:
        hotel_df.loc[idx, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_df.loc[idx, "Hotel Address"] = name_address["results"][0]["vicinity"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        

In [11]:
# Visualize to confirm hotels data appears
hotel_df

,City,Country,Lat,Lng,Hotel Name,Hotel Address
147,Yazman,PK,29.1212,71.7446,Royal Nawab Marriage Hall,"Rehman City, Yazman Mandi"
192,Bubaque,GW,11.2833,-15.8333,Le Mango Lodge,"Arquipélago dos Bijagós, Rua comercial"
327,Sur,OM,22.5667,59.5289,Sur Plaza Hotel,Sur
428,Nizwá,OM,22.9333,57.5333,Albayan Farm House,21 نزوى OM
516,Pokaran,IN,26.9167,71.9167,The Fort Pokaran - A Heritage Hotel,"Dist, Pokhran"
538,Turbat,PK,26.0023,63.0440,Shapur Hotel,"Main Rd, Turbat"


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Customize the size of the figure
figure_layout = {
    'width': '600px',  
    'height': '420px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout, zoom_level=2, center=(10,25))


In [14]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info, display_info_box=True)


# Add the heatmap layer to the map and add marker layer ontop of heat map
fig.add_layer(heatmap_layer)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='420px', margin='0 auto 0 auto', padding='1px', wi…